## Population-weighted exposure by race/ethnicity

In [12]:
"""
Load in data and calculate exposure ratio
"""

import pandas as pd

results_dir0 = "/content/drive/MyDrive/gpDept-ResearchDept/LNG Air Pollution/LNG Health - COBRA project/Version 5 analysis"
f1 = '/pop_weighted_exposure.results/pop_weighted_exposure.results.race_ethnicity2.b2030-a2030.xlsx'
f2 = '/pop_weighted_exposure.results/pop_weighted_exposure.results.poverty.b2023-a2028.xlsx'

which = 'race'
compare_to = 'pWhite'

# which = 'pov'
# compare_to = 'pAbovePovertyLevel'

if which == 'race':
    f = f1
elif which == 'pov':
    f = f2

sheets = ["all_projects_{}".format(which), "all_projects_LA_{}".format(which), "all_projects_TX_{}".format(which)]
dfs = []

for sheet in sheets:
    dfs.append(pd.read_excel(results_dir0 + f, sheet))

for df in dfs:
    for p in df['Pollutant'].unique():
        filtered = df[df['Pollutant'] == p]
        denom = filtered[filtered['Group'] == compare_to]['Exposure'].iloc[0]
        diff = filtered['Exposure']/denom - 1
        df.loc[filtered.index, 'Exposure_ratio'] = diff


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
pip install kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.0 MB/s eta 0:00:00


In [ ]:
"""
Create 2-part visualization for all projects
"""

import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px


# Sample dataframe
scope = 'LA'
plot_var = 'Exposure_ratio' # Exposure or Exposure_ratio

# Plot labels
if scope == 'US':
    df = dfs[0].copy(deep=True)
elif scope == 'TX':
    df = dfs[2].copy(deep=True)
elif scope == 'LA':
    df = dfs[1].copy(deep=True)
else:
    raise ValueError("Invalid scope")

# Filter out the pWhite group
if plot_var == 'Exposure_ratio':
    df_filtered = df
    df_filtered = df[df['Group'] != 'pWhite']
else:
    df_filtered = df
    df_filtered['Exposure'] = -df_filtered['Exposure']


# Create the figure
if plot_var == 'Exposure_ratio':
    fig = make_subplots(rows=1, cols=1)
else:
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Delta PM 2.5", "Delta O3"))

# Unique groups
groups = df_filtered['Group'].unique()
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink']

# Pollutants
pollutants = df_filtered['Pollutant'].unique()

# Add bars for each pollutant
for i, pollutant in enumerate(pollutants, start=1):
    df_pollutant = df_filtered[df_filtered['Pollutant'] == pollutant]
    solid_fill = (pollutant == pollutants[0])

    if plot_var == 'Exposure_ratio':
        i = 1
    else:
        i = i

    fig.add_trace(
        go.Bar(
            x=df_pollutant['Group'],
            y=df_pollutant[plot_var],
            name=f"PM 2.5" if solid_fill else "Ozone",
            marker=dict(
                color=colors[:len(df_pollutant)],
                pattern_shape="" if solid_fill else "/"
            ), showlegend=False
        ),
    row=i, col=1
    )

if plot_var == 'Exposure_ratio':
    fig.layout.yaxis.tickformat = ',.0%'

if plot_var == 'Exposure_ratio':
    if compare_to == 'pWhite':
        subtxt = 'white population'
    elif compare_to == 'pAbovePovertyLevel':
        subtxt = 'above poverty line population'
    else:
        raise ValueError("Invalid compare_to")

    if scope == 'US':
        title = 'Population-weighted exposure relative to {} (country-wide)'.format(subtxt)
    else:
        title = "Population-weighted exposure relative to in-state {} ({})".format(subtxt, scope)
else:
    if scope == 'US':
        title = 'Population-weighted exposure (country-wide)'
    else:
        title = "Population-weighted exposure ({})".format(scope)

# Update layout
fig.update_layout(
    title=title,
    # barmode='group',
    xaxis_title="Race/Ethnicity",
    yaxis_title="",
    height=600,
    yaxis = dict(range=[-.5, .7])
)

# Show the figure
fig.show()


In [17]:
"""
Create 2-part visualization for all projects
"""

import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

scopes = ['US', 'TX', 'LA']
fig = make_subplots(rows=4, cols=1,
                    subplot_titles=['National',
                                    'Texas only',
                                    'Louisiana only'],
                    vertical_spacing=0.07
                    )

for ii, sc in enumerate(scopes, start=1):

    # Sample dataframe
    scope = sc
    plot_var = 'Exposure_ratio' # Exposure or Exposure_ratio

    # Plot labels
    if scope == 'US':
        df = dfs[0].copy(deep=True)
    elif scope == 'TX':
        df = dfs[2].copy(deep=True)
    elif scope == 'LA':
        df = dfs[1].copy(deep=True)
    else:
        raise ValueError("Invalid scope")

    # Filter out the pWhite group
    if plot_var == 'Exposure_ratio':
        df_filtered = df
        df_filtered = df[(df['Group'] != 'pWhite') & (df['Group'] != 'pOther')]
    else:
        df_filtered = df
        df_filtered['Exposure'] = -df_filtered['Exposure']

    print(df_filtered['Group'].unique())
    df_filtered['Group'] = pd.Categorical(df_filtered['Group'], categories=['pBlackAA', 'pHispanicLatino',
                                                                            'pAsian', 'pAmerIndianAN',
                                                                            'pNativeHawaiianPI'])
    df_filtered = df_filtered.sort_values('Group')

    # Unique groups
    groups = df_filtered['Group'].unique()
    colors = ['#d54400', '#008294', '#89d2d8', '#508142', '#edc948']

    # Pollutants
    pollutants = df_filtered['Pollutant'].unique()

    # Add bars for each pollutant
    for i, pollutant in enumerate(pollutants, start=1):
        df_pollutant = df_filtered[df_filtered['Pollutant'] == pollutant]
        solid_fill = (pollutant == pollutants[0])

        if plot_var == 'Exposure_ratio':
            i = 1
        else:
            i = i

        fig.add_trace(
            go.Bar(
                x=df_pollutant['Group'],
                y=df_pollutant[plot_var]+1,
                name=f"PM 2.5" if solid_fill else "Ozone",
                marker=dict(
                    color=colors[:len(df_pollutant)],
                    pattern_shape="" if solid_fill else "/"
                ), showlegend=False
            ),
        row=ii, col=1
        )

fig.add_trace(
    go.Bar(
        x=[None],
        y=[None],
        name="PM 2.5",
        marker=dict(
            color="darkgrey",
            pattern_shape=""
        )
    ),
    row=4, col=1
)
fig.add_trace(
    go.Bar(
        x=[None],
        y=[None],
        name="Ozone",
        marker=dict(
            color="darkgrey",
            pattern_shape="/"
        )
    ),
    row=4, col=1
)

import numpy as np
# Update layout
fig.update_layout(
    title="Population-weighted exposure to LNG air pollutants",
    # barmode='group',
    # xaxis_title="Race/Ethnicity",
    yaxis_title="",
    height=1200,
    width=600,
    yaxis = dict(range=[0.4, 1.8], tickformat = ',.0%', title='Exposure rel. to white Americans'),
    yaxis2 = dict(range=[0.6, 1.3], tickformat = ',.0%', title='Exposure rel. to white Texans'),
    yaxis3 = dict(range=[0.6, 1.3], tickformat = ',.0%', title='Exposure rel. to white Louisianans'),
    xaxis = dict(range=[-0.65, 4.65],
                 ticktext=['', '', '', '', ''], tickvals=np.arange(5), tickmode='array'),
    xaxis2= dict(ticktext=['', '', '', '', ''], tickvals=np.arange(5), tickmode='array'),
    xaxis3= dict(ticktext=['Black/<br>African<br>American', 'Hispanic/<br>Latino', 'Asian', 'American<br>Indian/<br>Alaskan<br>Native', 'Native<br>Hawaiian/<br>Pacific<br>Islander'],
                 tickvals=np.arange(5),
                 tickmode='array', tickangle=0),
    shapes=[{'type': 'line','y0':1,'y1': 1, 'x0':-0.65, 'x1':4.65,
             'xref':'x1','yref':'y1',
                              'line': {'color': 'black','width': 1}},
            {'type': 'line','y0':1,'y1': 1, 'x0':-0.65, 'x1':4.65,
             'xref':'x2','yref':'y2',
                              'line': {'color': 'black','width': 1}},
            {'type': 'line','y0':1,'y1': 1, 'x0':-0.65, 'x1':4.65,
             'xref':'x3','yref':'y3',
                              'line': {'color': 'black','width': 1}}],
    legend = dict(
                            yanchor="top",
                            y=0.97,
                            xanchor="right",
                            x=0.96
                        )
)

fig.write_image(results_dir0 + "/pop_weighted_exposure.results/by_race_ethnicity.svg", engine="kaleido")
fig.write_image(results_dir0 + "/pop_weighted_exposure.results/by_race_ethnicity.pdf", engine="kaleido")
fig.write_image(results_dir0 + "/pop_weighted_exposure.results/by_race_ethnicity.jpg")

# Show the figure
fig.show()


['pBlackAA' 'pAmerIndianAN' 'pAsian' 'pNativeHawaiianPI' 'pHispanicLatino']
['pBlackAA' 'pAmerIndianAN' 'pAsian' 'pNativeHawaiianPI' 'pHispanicLatino']
['pBlackAA' 'pAmerIndianAN' 'pAsian' 'pNativeHawaiianPI' 'pHispanicLatino']


<ipython-input-17-bb2a96c0f045>:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-17-bb2a96c0f045>:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-17-bb2a96c0f045>:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

